In [ ]:
import Pkg
Pkg.activate(@__DIR__)

In [ ]:
using CairoMakie, ModelingToolkit, OrdinaryDiffEq, LaTeXStrings, Statistics, Printf

# Introduction to Neuroscience in Julia

# Workshop Module 1: Nernst Potential

![](https://cocalc.com/blobs/Basis_of_Membrane_Potential2.png?uuid=20ed11f4-efb0-481d-b48e-74075d31425f)

The Nernst equation describes the equilibrium potential for a single ion species across a membrane. It's fundamental to understanding how neurons generate and
maintain electrical signals.

The Nernst equation is:
E\_ion = \(RT/zF\) \* ln\(\[ion\]\_out / \[ion\]\_in\)

Where:

- E_ion: equilibrium potential for the ion (V)
- R: gas constant (8.314 J/(mol·K))
- T: temperature (K)
- z: valence of the ion
- F: Faraday constant (96,485 C/mol)
- [ion]_out, [ion]_in: ion concentrations outside and inside the cell



In [ ]:
# Constants
const R = 8.314      # Gas constant (J/(mol·K))
const F = 96485      # Faraday constant (C/mol)
const T = 310        # Body temperature (K, ~37°C)

# Nernst equation function
function nernst_potential(conc_out, conc_in, valence, temperature=T)
    """
    Calculate the Nernst potential for an ion
    
    Parameters:
    - conc_out: extracellular concentration (mM)
    - conc_in: intracellular concentration (mM)
    - valence: charge of the ion
    - temperature: temperature in Kelvin (default: 310K)
    
    Returns:
    - Equilibrium potential in mV
    """
    RT_over_F = (R * temperature) / F
    return 1000 * RT_over_F * log(conc_out / conc_in) / valence
end

In [ ]:
# Typical ion concentrations in mammalian neurons (mM)
println("\nTypical ion concentrations in mammalian neurons:")
println("Ion\t\tInside\t\tOutside\t\tValence")

# Potassium (K+)
K_in = 140.0   # mM
K_out = 5.0    # mM
K_valence = 1
E_K = nernst_potential(K_out, K_in, K_valence)
@printf("K+\t\t%.1f mM\t\t%.1f mM\t\t%+d\n", K_in, K_out, K_valence)

# Sodium (Na+)
Na_in = 10.0   # mM
Na_out = 145.0 # mM
Na_valence = 1
E_Na = nernst_potential(Na_out, Na_in, Na_valence)
@printf("Na+\t\t%.1f mM\t\t%.1f mM\t\t%+d\n", Na_in, Na_out, Na_valence)

# Chloride (Cl-)
Cl_in = 10.0   # mM
Cl_out = 110.0 # mM
Cl_valence = -1
E_Cl = nernst_potential(Cl_out, Cl_in, Cl_valence)
@printf("Cl-\t\t%.1f mM\t\t%.1f mM\t\t%+d\n", Cl_in, Cl_out, Cl_valence)

println("\nCalculated Nernst potentials:")
@printf("E_K  = %.1f mV\n", E_K)
@printf("E_Na = %.1f mV\n", E_Na)
@printf("E_Cl = %.1f mV\n", E_Cl)

## MODULE 2: THE HODGKIN-HUXLEY MODEL
The Hodgkin-Huxley model describes the ionic mechanisms underlying the action
potential in the squid giant axon. It consists of:

1. The membrane equation: $C_{m} \frac{dV}{dt} = I_{ext} - I_{Na} - I_{K} - I_{L}$
2. Sodium current: $I_{Na} = g_{Na} m^3 h (V - E_{Na})$
3. Potassium current: $I_{K} = g_{K} * n^4 (V - E_{K})$
4. Leak current: $I_{L} = g_{L} (V - E_{L})$
5. Gating variable dynamics: dx/dt = $\alpha_{x}(V) * (1-x) - β_{x}(V) * x$

Where m, h, n are gating variables for Na+ activation, Na+ inactivation, 
and K+ activation respectively.

In [ ]:
# Hodgkin-Huxley parameters (original squid axon values)
struct HHParams
    # Membrane capacitance (µF/cm²)
    C_m::Float64
    
    # Maximum conductances (mS/cm²)
    g_Na::Float64
    g_K::Float64
    g_L::Float64
    
    # Reversal potentials (mV)
    E_Na::Float64
    E_K::Float64
    E_L::Float64
    
    HHParams() = new(1.0, 120.0, 36.0, 0.3, 50.0, -77.0, -54.387)
end

# Create parameter set
params = HHParams()

println("Hodgkin-Huxley Model Parameters:")
println("Membrane capacitance: $(params.C_m) µF/cm²")
println("Na+ conductance: $(params.g_Na) mS/cm²")
println("K+ conductance: $(params.g_K) mS/cm²")
println("Leak conductance: $(params.g_L) mS/cm²")
println("Na+ reversal potential: $(params.E_Na) mV")
println("K+ reversal potential: $(params.E_K) mV")
println("Leak reversal potential: $(params.E_L) mV")

In [ ]:
# Rate functions for gating variables
function alpha_m(V)
    if abs(V + 40) < 1e-6
        return 1.0
    else
        return 0.1 * (V + 40) / (1 - exp(-(V + 40) / 10))
    end
end

function beta_m(V)
    return 4.0 * exp(-(V + 65) / 18)
end

function alpha_h(V)
    return 0.07 * exp(-(V + 65) / 20)
end

function beta_h(V)
    return 1.0 / (1 + exp(-(V + 35) / 10))
end

function alpha_n(V)
    if abs(V + 55) < 1e-6
        return 0.1
    else
        return 0.01 * (V + 55) / (1 - exp(-(V + 55) / 10))
    end
end

function beta_n(V)
    return 0.125 * exp(-(V + 65) / 80)
end

# Steady-state values and time constants
m_inf(V) = alpha_m(V) / (alpha_m(V) + beta_m(V))
h_inf(V) = alpha_h(V) / (alpha_h(V) + beta_h(V))
n_inf(V) = alpha_n(V) / (alpha_n(V) + beta_n(V))

tau_m(V) = 1 / (alpha_m(V) + beta_m(V))
tau_h(V) = 1 / (alpha_h(V) + beta_h(V))
tau_n(V) = 1 / (alpha_n(V) + beta_n(V))

# Plotting gating variable steady-states and time constants

In [ ]:
V_range = -100:1:50

# Calculate steady-state values
m_steady = [m_inf(V) for V in V_range]
h_steady = [h_inf(V) for V in V_range]
n_steady = [n_inf(V) for V in V_range]

# Calculate time constants
tau_m_vals = [tau_m(V) for V in V_range]
tau_h_vals = [tau_h(V) for V in V_range]
tau_n_vals = [tau_n(V) for V in V_range]

# Plot steady-state values
fig3 = Figure()
ax3 = Axis(fig3[1, 1],
    xlabel = "Membrane Potential (mV)",
    ylabel = "Steady-state Value",
    title = "Gating Variable Steady States")

lines!(ax3, V_range, m_steady, label = "m∞ (Na⁺ activation)", linewidth = 3)
lines!(ax3, V_range, h_steady, label = "h∞ (Na⁺ inactivation)", linewidth = 3)
lines!(ax3, V_range, n_steady, label = "n∞ (K⁺ activation)", linewidth = 3)

axislegend(ax3, position = :rt)
display(fig3)

In [ ]:
fig4 = Figure()
ax4 = Axis(fig4[1, 1],
    xlabel = "Membrane Potential (mV)",
    ylabel = "Time Constant (ms)",
    title = "Gating Variable Time Constants",
    yscale = log10)

lines!(ax4, V_range, tau_m_vals, label = "τₘ (Na⁺ activation)", linewidth = 3)
lines!(ax4, V_range, tau_h_vals, label = "τₕ (Na⁺ inactivation)", linewidth = 3)
lines!(ax4, V_range, tau_n_vals, label = "τₙ (K⁺ activation)", linewidth = 3)

axislegend(ax4, position = :rt)
display(fig4)

# Building Hodgkin-Huxley model with ModelingToolkit

In [ ]:
# Define the symbolic variables and parameters using ModelingToolkit
@independent_variables t
@variables V(t) m(t) h(t) n(t)
@parameters C_m g_Na g_K g_L E_Na E_K E_L I_ext

# Create the differential operator
D = Differential(t)

# Define the rate functions as symbolic expressions

# Alpha and beta functions for gating variables
α_m = ifelse(abs(V + 40) < 1e-6, 1.0, 0.1 * (V + 40) / (1 - exp(-(V + 40) / 10)))
β_m = 4.0 * exp(-(V + 65) / 18)
    
α_h = 0.07 * exp(-(V + 65) / 20)
β_h = 1.0 / (1 + exp(-(V + 35) / 10))
    
α_n = ifelse(abs(V + 55) < 1e-6, 0.1, 0.01 * (V + 55) / (1 - exp(-(V + 55) / 10)))
β_n = 0.125 * exp(-(V + 65) / 80)

# Ionic currents
I_Na = g_Na * m^3 * h * (V - E_Na)
I_K = g_K * n^4 * (V - E_K)
I_L = g_L * (V - E_L)

# Differential equations
eqs = [
    D(V) ~ (I_ext - I_Na - I_K - I_L) / C_m,
    D(m) ~ α_m * (1 - m) - β_m * m,
    D(h) ~ α_h * (1 - h) - β_h * h,
    D(n) ~ α_n * (1 - n) - β_n * n
]


# Create the system
@named hh_system = ODESystem(eqs,t)

# Simplify the system
hh_system_simplified = structural_simplify(hh_system)

# Define parameter values
param_values = [
    C_m => 1.0,      # µF/cm²
    g_Na => 120.0,   # mS/cm²  
    g_K => 36.0,     # mS/cm²
    g_L => 0.3,      # mS/cm²
    E_Na => 50.0,    # mV
    E_K => -77.0,    # mV
    E_L => -54.387,  # mV
    I_ext => 10.0    # µA/cm²
]

# Initial conditions (resting state)
V_rest = -65.0  # mV
m0 = m_inf(V_rest)
h0 = h_inf(V_rest)
n0 = n_inf(V_rest)

initial_conditions = [
    V => V_rest,
    m => m0,
    h => h0,
    n => n0
]

# Create and solve the ODE problem
tspan = (0.0, 50.0)  # 50 ms simulation
prob = ODEProblem(hh_system_simplified, initial_conditions, tspan, param_values)

sol = solve(prob, Tsit5(), saveat=0.01)

# Extract solutions for plotting
times = sol.t
V_sol = sol[V]
m_sol = sol[m]
h_sol = sol[h]
n_sol = sol[n]

# Plot the action potential
fig5 = Figure(;size = (1000, 600))
ax5 = Axis(fig5[1, 1],
    xlabel = "Time (ms)",
    ylabel = "Membrane Potential (mV)",
    title = "Hodgkin-Huxley Action Potential (I = 10 µA/cm²)")

lines!(ax5, times, V_sol, linewidth = 4, color = :blue)
hlines!(ax5, [V_rest], linestyle = :dash, color = :gray, linewidth = 2, label = "Resting potential")

axislegend(ax5, position = :rt)
display(fig5)

In [ ]:
# Plot gating variables over time
fig6 = Figure(;size = (1000, 600))
ax6 = Axis(fig6[1, 1],
    xlabel = "Time (ms)",
    ylabel = "Gating Variable",
    title = "Gating Variables During Action Potential")

lines!(ax6, times, m_sol, label = "m (Na⁺ activation)", linewidth = 3)
lines!(ax6, times, h_sol, label = "h (Na⁺ inactivation)", linewidth = 3)
lines!(ax6, times, n_sol, label = "n (K⁺ activation)", linewidth = 3)

axislegend(ax6, position = :rt)
display(fig6)

In [ ]:
# Calculate and plot ionic currents
I_Na_vals = [-params.g_Na * m_sol[i]^3 * h_sol[i] * (V_sol[i] - params.E_Na) for i in 1:length(times)]
I_K_vals = [-params.g_K * n_sol[i]^4 * (V_sol[i] - params.E_K) for i in 1:length(times)]
I_L_vals = [-params.g_L * (V_sol[i] - params.E_L) for i in 1:length(times)]

fig7 = Figure(;size = (1000, 600))
ax7 = Axis(fig7[1, 1],
    xlabel = "Time (ms)",
    ylabel = "Current (µA/cm²)",
    title = "Ionic Currents During Action Potential")

lines!(ax7, times, I_Na_vals, label = "I_Na", linewidth = 3)
lines!(ax7, times, I_K_vals, label = "I_K", linewidth = 3)
lines!(ax7, times, I_L_vals, label = "I_leak", linewidth = 3)
hlines!(ax7, [0], linestyle = :dash, color = :black, alpha = 0.5)

axislegend(ax7, position = :rt)
display(fig7)